# Step 1: Set up D3 area

In [1]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [2]:
%%javascript
require.config({paths: {d3: "https://d3js.org/d3.v4.min"}});
require(['d3'], function(d3) {
    window.d3 = d3;
})

<IPython.core.display.Javascript object>

In [3]:
html_template = Template('''
<svg id="graph-div"></div>
<script> $js_text </script>
''')

In [17]:
js_text_template = Template('''
var data = $data;
var itemWidth = 50;
var itemHeight = 50;

var chart = d3.select('#graph-div')
    .attr('width', 800)
    .attr('height', itemHeight*4);

function update() {
    var row = chart.selectAll('.item')
        .data(data);
    
    var row2 = row.enter()
        .append('g')
        .attr("class", "item");
        
    row2.merge(row)
        .attr('transform', function(d, i){
            return 'translate(' + i*(itemWidth+5) + ', ' + itemHeight + ')rotate(' + d + ', ' + itemWidth*0.5 + ', ' + itemHeight*0.5 + ')';
        });

    row2.append('rect')
        .attr("width", itemWidth)
        .attr("height", itemHeight)
        .attr("fill", "#9999FF")
        .attr("stroke", "#5555AA");

    row2.append("text")
        .text("Hello");
}

update();
''')

In [18]:
data = [0,10,20,30];
js_text = js_text_template.substitute({'data': json.dumps(data)})
HTML(html_template.substitute({'js_text': js_text}))

Only run the following cell after running everything else.  It was only places here so the slider could be next to the visualization.

In [36]:
interact(update_from_slider, x=widgets.IntSlider(min=0,max=len(data)-10,step=1,value=0));

### OK, the D3 area is set up

Now we'll focus on live updating.  A manual test first.

In [25]:
js_text_template_2 = Template('''
data = $data;
update();
console.log("updating");
''')

In [26]:
data = [0,0,0,0,0,0]

def update_graph(data):
    js_text = js_text_template_2.substitute({'data': json.dumps(data)})
    display(HTML('<script>' + js_text + '</script>'))

update_graph(data)

# Step 2: Now use MQTT to update the graph

Now for the fun stuff.  Using the `update_graph(data)` function set up above.  Each group of data, separated by `"start"` messages, is saved in the `tests` array.  As new data comes in, the live data graph is updated to show data from the last test.  All data is neatly saved inside the tests array for replotting and further analysis later.

Using the CloudMQTT free Cat plan:
* https://api.cloudmqtt.com/sso/cloudmqtt/console
* haipjacob@gmail.com:paramour-sieve-paper
* If the Websocket UI has trouble connecting, restart the instance

In [29]:
import paho.mqtt.client as mqtt

n_to_save = 10
data = [0 for i in range(n_to_save)]

# update_graph(data)

def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("/outTopic")

def on_message(client, userdata, msg):
    global data
    try:
        msg_json = json.loads(msg.payload)
    except:
        print("Error")
        print(msg.topic+" "+str(msg.payload))
        return
    print(msg_json)
    if msg_json['type'] == "BINARY" and msg_json['label'] == "Or":
        # data.append((msg_json['X'], msg_json['Y'], msg_json['Z']))
        data.append(msg_json['Z'])
        update_graph(data[-n_to_save:])
        print(data[-6:])

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.username_pw_set("zettlmtm", "VOUbRcmhjffA")
client.connect("m11.cloudmqtt.com", 19280, 60)
client.loop_start()
# Make sure to call client.loop_stop() later

Connected with result code 0
{u'Z': 1.19, u'label': u'Or', u'Y': 0.0, u'X': 348.12, u'tick': u'2045918', u'type': u'BINARY'}


[0, 0, 0, 0, 0, 1.19]
{u'Z': 1.19, u'label': u'Or', u'Y': 0.0, u'X': 348.12, u'tick': u'2046122', u'type': u'BINARY'}


[0, 0, 0, 0, 1.19, 1.19]
{u'Z': 1.19, u'label': u'Or', u'Y': 0.0, u'X': 348.12, u'tick': u'2046327', u'type': u'BINARY'}


[0, 0, 0, 1.19, 1.19, 1.19]
{u'Z': 1.19, u'label': u'Or', u'Y': 0.0, u'X': 348.12, u'tick': u'2046530', u'type': u'BINARY'}


[0, 0, 1.19, 1.19, 1.19, 1.19]
{u'Z': 1.19, u'label': u'Or', u'Y': 0.0, u'X': 348.12, u'tick': u'2046735', u'type': u'BINARY'}


...

[1.62, 1.62, 1.62, 1.62, 1.62, 1.62]
{u'Z': 1.62, u'label': u'Or', u'Y': 0.06, u'X': 349.31, u'tick': u'2060425', u'type': u'BINARY'}


[1.62, 1.62, 1.62, 1.62, 1.62, 1.62]
{u'Z': 1.56, u'label': u'Or', u'Y': 0.0, u'X': 349.31, u'tick': u'2060629', u'type': u'BINARY'}


[1.62, 1.62, 1.62, 1.62, 1.62, 1.56]
{u'Z': 1.56, u'label': u'Or', u'Y': 0.0, u'X': 349.31, u'tick': u'2060835', u'type': u'BINARY'}


[1.62, 1.62, 1.62, 1.62, 1.56, 1.56]
{u'Z': 1.5, u'label': u'Or', u'Y': 0.0, u'X': 349.31, u'tick': u'2061038', u'type': u'BINARY'}


[1.62, 1.62, 1.62, 1.56, 1.56, 1.5]


In [31]:
client.loop_stop()

3

In [33]:
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.19, 1.19, 1.19, 1.19, 1.19, 1.19, 1.19, 1.19, 1.19, 1.19, 1.19, 1.19, 1.19, 0.69, -3.31, -13.19, -20.94, -27.94, -35.81, -38.56, -39.88, -42.94, -44.19, -45.75, -42.88, -30.56, -17.06, -9.56, -3.75, 2.0, 7.56, 19.44, 32.81, 43.81, 57.19, 69.88, 79.25, 88.88, 91.06, 91.12, 89.0, 84.75, 76.69, 66.81, 56.94, 47.31, 38.5, 30.38, 22.88, 10.81, 4.06, 3.56, 2.12, 2.06, 1.94, 1.88, 1.81, 1.75, 1.75, 1.69, 1.62, 1.62, 1.62, 1.62, 1.62, 1.62, 1.62, 1.62, 1.62, 1.62, 1.62, 1.62, 1.56, 1.56, 1.5, 1.5]


In [19]:
from time import sleep
for i in range(len(data)-10):
    update_graph(data[i:i+10])
    sleep(0.2)

# Replaying old data!

The code in the cell below replays the old data on the graph using the collected data.  This could also be a Jupyter widget to allow scrubbing if I knew how that worked.

In [34]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [35]:
def update_from_slider(x):
    update_graph(data[x:x+10])

In [18]:
interact(update_from_slider, x=widgets.IntSlider(min=0,max=len(data)-10,step=1,value=0));